<a href="https://colab.research.google.com/github/KRUTHIKA2405/Green-AI/blob/main/Forest_Fire_Detection_Using_Satellite_Imagery.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1 style="text-align: center;">Forest Fire Detection using Computer Vision</h1>


![forest%20fire%20cover%20image.png](attachment:forest%20fire%20cover%20image.png)

## 1. Problem Statement

Imagine you are in charge of watching a big forest. You want to keep everyone safe, but sometimes wildfires can start without warning. These fires can spread quickly and cause a lot of damage. So, we want to build a computer program that can look at pictures of forests and tell us if there’s a fire starting or not. This program can help people react faster to stop wildfires!

## 2. Objectives

This project will:

    Teach the computer to recognize pictures with wildfires and without wildfires.
    Build a model (a smart program) that looks at forest pictures and decides if there’s a fire.
    Show us a simple screen (a GUI) where we can upload forest pictures and get quick answers on fire danger.
    Help keep forests and people safe by giving early warnings about fires.

### 3. Dataset Information

To teach the computer about wildfires, we’ll use a special set of pictures called a dataset. This dataset has pictures of forests divided into two types: with a fire (wildfire) and without a fire (no wildfire). These images help the computer learn to tell the difference.

    Where to Get the Dataset: You can download the dataset from Kaggle
    https://www.kaggle.com/datasets/abdelghaniaaba/wildfire-prediction-dataset
    Dataset Folders:
        Inside, there are three main folders: train, test, and valid.
        Each folder has two subfolders:
            Wildfire: Contains pictures showing wildfires in forests.
            NoWildfire: Contains pictures showing forests without any fire.

## 4. Understanding the CNN Model

CNN stands for Convolutional Neural Network. It’s a special type of program that can look at pictures and learn to find important details. Imagine how you look at a picture and notice trees, animals, or fire — CNN does something similar!

Here’s how it works, step-by-step:

    Looking at Patterns: The CNN looks at lots of tiny patterns in pictures, like shapes and colors.
    Learning from Examples: It looks at many images of wildfires and no wildfires to learn what makes them different.
    Making Predictions: Once trained, the CNN can look at a new picture and guess if it shows a fire or not.

## 5. Code Implementation

### Step 1: Importing Libraries

We’ll use Python with TensorFlow/Keras for the CNN model and Tkinter for the GUI. Make sure you have these libraries installed.

In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator #for data augumentation
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
import tkinter as tk
from tkinter import filedialog #provides a dialog box to select files.
from PIL import Image, ImageTk #ImageTk - Converts images for display in Tkinter GUIs
import numpy as np

### Step 2: Loading and Preprocessing the Data

We'll use ImageDataGenerator to load images from your dataset's train, valid, and test folders. This will resize and normalize images so our model can use them.

In [ ]:
 Set up directories
train_dir = r"train"
valid_dir = r"valid"
test_dir = r"test"

# Set up ImageDataGenerators for loading images
train_datagen = ImageDataGenerator(rescale=1./255)
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# Load images from directories
train_generator = TRAIN_datagen.flow_from_directory(train_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
valid_generator = VALID_datagen.flow_from_directory(valid_dir, target_size=(64, 64), batch_size=32, class_mode='binary')
test_generator = TEST_datagen.flow_from_directory(test_dir, target_size=(64, 64), batch_size=32, class_mode='binary')

In [7]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# Define data augmentation for training data
TRAIN_datagen = ImageDataGenerator(
    rescale=1.0/255.0,
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode='nearest'
)

# Define data augmentation for validation and test data
VALID_datagen = ImageDataGenerator(rescale=1.0/255.0)
TEST_datagen = ImageDataGenerator(rescale=1.0/255.0)


### Step 3: Building the CNN Model

The CNN model will have multiple convolutional layers to extract features, followed by dense layers for classification.
- Sequential: When you need a simple feed-forward model with a single input and output.
- Functional: When your model has multiple inputs/outputs or non-linear architectures.
- Subclassing: When custom forward logic is required or your model's architecture cannot be easily represented using pre-defined layers.

In [ ]:
# Building a simple CNN model
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(64, 64, 3)),
    MaxPooling2D(pool_size=(2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Binary classification: wildfire or no wildfire
])

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


We noticed some images in our dataset are incomplete or corrupted, causing TensorFlow to fail when trying to load them.
Here are a few steps to handle this issue:

Solution: Skip Corrupted Images

One way to handle this is to modify the data loading process to skip over corrupted images.

    Re-import the PIL library with a setting that will ignore truncated images.

    We add this line at the beginning of your script.

In [ ]:
from PIL import ImageFile
ImageFile.LOAD_TRUNCATED_IMAGES = True

### Step 4: Training the Model

Train the model with the train and valid data.

In [ ]:
# Train the model
history = model.fit(train_generator, validation_data=valid_generator, epochs=10, verbose=1)

In [ ]:
#save the model
model.save("ffd_model.h5")

### Step 5: Building the GUI with Tkinter

Now, let’s create a GUI that lets us upload an image and predict if there’s a wildfire.

In [ ]:
import tkinter as tk
from tkinter import filedialog

from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf

from tensorflow.keras.models import load_model

In [ ]:
# Load the model
model = load_model("ffd_model.h5")
print("Model loaded successfully!")

In [ ]:
# Function to load and predict an image
def predict_image():
    # Open file dialog to select an image
    file_path = filedialog.askopenfilename()
    if file_path:
        # Display the image in the GUI
        img = Image.open(file_path)
        img = img.resize((200, 200))
        img = ImageTk.PhotoImage(img)    #convert image for tk
        image_label.configure(image=img) #update the image in GUI
        image_label.image = img

        # Preprocess the image for the model
        img_for_model = Image.open(file_path).resize((64, 64))
        img_array = np.array(img_for_model) / 255.0  # Rescale like during training
        img_array = np.expand_dims(img_array, axis=0)  # Add batch dimension

        # Make a prediction
        prediction = model.predict(img_array)[0][0] #extracts the scalar prediction value
        result = "Wildfire" if prediction > 0.5 else "No Wildfire"
        result_label.config(text="Prediction: " + result)

# Setting up the GUI window
root = tk.Tk()
root.title("Forest Fire Detection")
root.geometry("400x400")

# Add widgets
btn = tk.Button(root, text="Upload Image", command=predict_image) #button triggers the predict_image() function when clicked
btn.pack(pady=20)

#Placeholder for displaying the selected image
image_label = tk.Label(root)
image_label.pack()

#Label to display the prediction result
result_label = tk.Label(root, text="Prediction: ", font=("Helvetica", 16))
result_label.pack(pady=20)

#Starts the Tkinter event loop, keeping the GUI active until manually closed
root.mainloop()

### Step 6: Testing the Model

You can now use the GUI to test the model. Click "Upload Image," select an image, and the GUI will display the image and predict if it shows a wildfire or not.

## 5. Conclusion

With this project, we have created a simple computer program that helps spot wildfires early by looking at pictures of forests. This project shows how computer vision, a field that teaches computers to see and understand images, can help us make safer decisions for people and nature. Using CNN, we taught the computer to recognize fires, giving us a way to respond faster and prevent fires from spreading.

![forest%20fire%20image.png](attachment:forest%20fire%20image.png)